In [1]:
import pandas as pd
import numpy as np
import os
import glob

# Combine and format the transfermarkt data

Combine team files into one dataframe

In [2]:
files = glob.glob(os.path.join('data', 'transfermarkt', '*'))
team_dfs = []
for file in files:
    team_dfs.append(pd.read_parquet(file))
df_all_players = pd.concat(team_dfs)
df_all_players.reset_index(drop=True, inplace=True)

Player number from string to numeric

In [3]:
df_all_players['number'] = pd.to_numeric(df_all_players.number, errors='coerce')

# You might need to edit this. As the transfermarkt value format depends on your location (mine is in euros)

In [4]:
# you might need to localise these values
mask_thousand = df_all_players['market_value'].str[-3:] == 'Th.'
df_all_players['market_value'] = (df_all_players['market_value']
                                  .str.replace('€', '', regex=False)
                                  .str.replace('Th.', '', regex=False)
                                  .str.replace('m', '', regex=False))
df_all_players['market_value'] = pd.to_numeric(df_all_players['market_value'], errors='coerce')
df_all_players.loc[mask_thousand, 'market_value'] = df_all_players.loc[mask_thousand, 'market_value'] / 1000
df_all_players.rename({'market_value': 'market_value_euro_millions'}, axis='columns', inplace=True)

Format date/ height columns

In [5]:
df_all_players['dob'] = pd.to_datetime(df_all_players['dob_age'].str.split(pat='(').str[0], errors='coerce')
df_all_players['joined'] = pd.to_datetime(df_all_players['joined'], errors='coerce')
df_all_players['contract_expires'] = pd.to_datetime(df_all_players['contract_expires'], errors='coerce')
df_all_players.drop(['dob_age'], axis=1, inplace=True)
df_all_players['height'] = pd.to_numeric(df_all_players['height'].str.replace('m', '').str.replace(',','.'))

Save the data

In [6]:
df_all_players.to_parquet(os.path.join('data', 'players_transfermarket.parquet'))